# Finger Count using OpenCV and Contour Detection

In [18]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

In [17]:
# Global variables
background = None
accumulated_weight = 0.5
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [16]:
def calc_accum_avg(frame, accumulated_weight):
    global background
    if background is None:
        background = frame.copy().astype('float')
        return
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [15]:
def segment(frame, threshold=15):
    global background

    diff = cv2.absdiff(background.astype("uint8"), frame)

    _, thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

    kernel = np.ones((3, 3), np.uint8)
    thresholded = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)  # Fill small holes
    thresholded = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel)   # Remove small blobs
    contours, _ = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None
    hand_segment = max(contours, key=cv2.contourArea)
    if cv2.contourArea(hand_segment) < 1000:
        return None
    if cv2.countNonZero(thresholded) < 1500:
        return None


    return (thresholded, hand_segment)

In [21]:
def count_fingers(thresholded, hand_segment):
    hull = cv2.convexHull(hand_segment, returnPoints=False)
    defects = cv2.convexityDefects(hand_segment, hull)

    if defects is None:
        return 0

    finger_count = 0

    for i in range(defects.shape[0]):
        s, e, f, d = defects[i, 0]
        start = tuple(hand_segment[s][0])
        end = tuple(hand_segment[e][0])
        far = tuple(hand_segment[f][0])

        # Calculate length of triangle sides
        a = np.linalg.norm(np.array(end) - np.array(start))
        b = np.linalg.norm(np.array(far) - np.array(start))
        c = np.linalg.norm(np.array(end) - np.array(far))

        # Apply cosine rule to find angle
        angle = np.arccos((b**2 + c**2 - a**2)/(2*b*c))

        # Ignore defects with angle > 90 degrees or small depth
        if angle <= np.pi / 2 and d > 10000:
            finger_count += 1

    return min(finger_count + 1, 5)

In [22]:
# Start the camera
cam = cv2.VideoCapture(0)
num_frames = 0

while True:
    ret, frame = cam.read()
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]

    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    if num_frames < 60:
        calc_accum_avg(gray, accumulated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "WAIT! GETTING BACKGROUND AVG.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("Finger Count", frame_copy)
    else:
        hand = segment(gray)
        if hand is not None:
            thresholded, hand_segment = hand
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0), 1)
            fingers = count_fingers(thresholded, hand_segment)
            cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("Thresholded", thresholded)

    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0, 0, 255), 5)
    num_frames += 1
    cv2.imshow("Finger Count", frame_copy)

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()